<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Подготовка-к-обучению" data-toc-modified-id="Подготовка-к-обучению-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка к обучению</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LightGBM-model" data-toc-modified-id="LightGBM-model-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LightGBM model</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>LGBMClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**Заказчик** — Интернет-магазин «Викишоп».

**Цель исследования** 

* Определения токсичных комментариев в реальном времени для и их отправку на модерацию

**Входные данные**  — ```/toxic_comments.csv``` - 


**Результаты исследования** — Получена модель подходящая под требования заказчика (F1 >= 0.75)

## Подготовка

In [1]:
#общее
import pandas as pd
import numpy as np
import time
from tqdm import notebook
from tqdm import tqdm

#работа с текстом
#!pip install pymystem3
from pymystem3 import Mystem

from nltk.stem import WordNetLemmatizer 
import re 
import nltk

from nltk.corpus import stopwords as nltk_stopwords
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer 

#обучение
#!pip install torch
import torch

#!pip install transformers
import transformers

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('toxic_comments.csv')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


**Описание:**
* text — текст комментария
* toxic — целевой признак


### Изучение данных

Сразу видим, лишний столбец который скорее всего задублировал индекс. Так как мы будем работать только с коменнтариями и целевым признаком, можно сразу удалить

In [3]:
df = df.drop(['Unnamed: 0'], axis=1)

In [4]:
# изучим датафрейм
# функция для обработки данных датафрейма
def research(df):
    print('Информация по таблице:', '\n')
    df.info()
    print()
    print('Количество пропусков:', '\n')
    print(df.isna().sum(), '\n', '\n')
    print('Процент пропусков:', '\n')
    print(df.isna().mean()*100, '\n')

    display(df.describe(include='all'))
    print()
    print('Число явных дубликатов:', df.duplicated().sum(), '\n')

In [5]:
research(df)

Информация по таблице: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB

Количество пропусков: 

text     0
toxic    0
dtype: int64 
 

Процент пропусков: 

text     0.0
toxic    0.0
dtype: float64 



,text,toxic
count,159292,159292.000000
unique,159292,NaN
top,Explanation\nWhy the edits made under my usern...,NaN
freq,1,NaN
mean,NaN,0.101612
std,NaN,0.302139
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,0.000000



Число явных дубликатов: 0 



In [6]:
#Глянем, что по балансу
display(df['toxic'].value_counts())

0    143106
1     16186
Name: toxic, dtype: int64

**Промежуточный вывод** 

Можно обратить внимание, что баланса нет, токсичных коментариев явно меньше. Стоит это учесть.
В остальном данные нормальные (кроме комментариев, их обычными методами не проверить)
* пропусков нет 
* типы столбцов подходящие
* есть необходимые данные для работы


### Подготовка к обучению

In [7]:
#для начала очистим текст

def clear_text(text):
    clear = re.sub(r'[^a-zA-Z ]', ' ', text)
    clear_text = " ".join(clear.split()) 
    return clear_text

In [8]:
df['clear_text'] = df['text'].apply(clear_text)

df.head()

,text,toxic,clear_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [9]:
# очистка с прогресс баром
''' 
for i in tqdm (range (100), 
               desc="Loading…", 
               ascii=False, ncols=75):
    df['clear_text'] = df['text'].apply(clear_text)

df.head() 
'''

' \nfor i in tqdm (range (100), \n               desc="Loading…", \n               ascii=False, ncols=75):\n    df[\'clear_text\'] = df[\'text\'].apply(clear_text)\n\ndf.head() \n'

Текст подчистили, выглядит явно лучше, можно двигаться дальше.

In [10]:
#Далее на очереде лематизация, данный код очень долго выпоняется поэтому посмотрим другие варианты
'''
def lemmatize(text):
    m = Mystem()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
        
    return lemm_text
'''

'\ndef lemmatize(text):\n    m = Mystem()\n    lemm_list = m.lemmatize(text)\n    lemm_text = "".join(lemm_list)\n        \n    return lemm_text\n'

In [11]:
m = WordNetLemmatizer()

def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    
    return ' '.join([m.lemmatize(w) for w in word_list])

In [12]:
df['lemm_text'] = df['clear_text'].apply(lemmatize)
df.head()


,text,toxic,clear_text,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...,You sir are my hero Any chance you remember wh...


In [13]:
#Оставим только очищенный текст и целевой признак
df_clear = df.drop(['text','clear_text'], axis=1)
df_clear

,toxic,lemm_text
0,0,Explanation Why the edits made under my userna...
1,0,D aww He match this background colour I m seem...
2,0,Hey man I m really not trying to edit war It s...
3,0,More I can t make any real suggestion on impro...
4,0,You sir are my hero Any chance you remember wh...
...,...,...
159287,0,And for the second time of asking when your vi...
159288,0,You should be ashamed of yourself That is a ho...
159289,0,Spitzer Umm there no actual article for prosti...
159290,0,And it look like it wa actually you who put on...


Создадим корпус постов. Преобразуем столбец `'lemm_text'` в список текстов. 

In [14]:
#подготовка корпуса
corpus = df_clear['lemm_text'].values

#Перевод в кодировку Unicode U убивает ядро и в целом нет смысла приводить английские слова
#corpus = df_clear['lemm_text'].values.astype('U')

In [15]:
#подготовка обучающих выборок
features = corpus
target = df_clear['toxic'].values

train_features, test_features, train_target, test_target = train_test_split(features, 
                                                                            target, test_size=0.2, random_state=12345)

In [16]:
stopwordss = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words = stopwordss)
train_features = count_tf_idf.fit_transform(train_features)

In [17]:
test_features = count_tf_idf.transform(test_features)

**Промежуточный итог:**
    
Данные проверены, подготволены для дальнешего обучения.

## Обучение

In [18]:
# создадим табличку в которую будем записывать результаты
result_test = {}

### LogisticRegression

In [19]:
lr_model = LogisticRegression()

hyperparams = [{'C':[0.1, 1, 3, 10],
                'class_weight':['balanced']}]
clf = GridSearchCV(lr_model, hyperparams, scoring='f1',cv=3)
clf.fit(train_features, train_target)
print("Лучшие параметры модели:", '\n')

LR_best_params = clf.best_params_
print(LR_best_params, '\n')

print('F1:', clf.best_score_)

Лучшие параметры модели: 

{'C': 10, 'class_weight': 'balanced'} 

F1: 0.7596793158493892


Выбираем модель со следующими показателями:

{'C': 10, 'class_weight': 'balanced'}

И наилучшим результатом -

F1: 0.759513569910502

### LightGBM model

In [20]:
%%time

lgbm_model = LGBMClassifier()
for i in [50, 100, 200, 500, 1000]:
    hyperparams = [{'max_depth' : [1],
                    'learning_rate':[0.1],
                    'n_estimators' : [i],
                    'random_state':[12345]}]
    clf = GridSearchCV(lgbm_model, hyperparams, scoring='f1',cv=3)
    clf.fit(train_features, train_target)
    print("Параметры модели:", '\n')
    LGBM_best_params = clf.best_params_
    print(LGBM_best_params, '\n')
    print('F1:', clf.best_score_)

print('\n', 'Затрачиваемое время:')

Параметры модели: 

{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 50, 'random_state': 12345} 

F1: 0.346571316297958
Параметры модели: 

{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 100, 'random_state': 12345} 

F1: 0.4212532296246884
Параметры модели: 

{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 200, 'random_state': 12345} 

F1: 0.5048408371574884
Параметры модели: 

{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 500, 'random_state': 12345} 

F1: 0.6109712552531262
Параметры модели: 

{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 1000, 'random_state': 12345} 

F1: 0.6650633125457543

 Затрачиваемое время:
CPU times: total: 21min 24s
Wall time: 7min 9s


Выбираем модель со следующими показателями:

{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 500, 'random_state': 12345} 

И наилучшим результатом -
F1: 0.6109712552531262

### CatBoostClassifier

In [21]:
%%time
#при создании массива убивается ядро
#train_featurescat =  train_features.toarray() 

cbr_model = CatBoostClassifier(eval_metric="F1", 
                                   iterations=100, 
                                   max_depth=5, 
                                   learning_rate=0.9, 
                                   random_state=42)
cbr_model.fit(train_features, train_target, verbose=20)


predictions_train = cbr_model.predict(train_features)
f1_train = f1_score(train_target, predictions_train)

predictions_test = cbr_model.predict(test_features)
f1_test = f1_score(test_target, predictions_test)

print()
print("F1 обучающей выборки:", f1_train)
print("F1 тестовой выборки: ", f1_test)
print('Матрица ошибок')
print(confusion_matrix(test_target, predictions_test))
print('\n', 'Время обработки:')

0:	learn: 0.3869713	total: 2.06s	remaining: 3m 24s
20:	learn: 0.7257268	total: 29.7s	remaining: 1m 51s
40:	learn: 0.7622340	total: 56.6s	remaining: 1m 21s
60:	learn: 0.7812731	total: 1m 23s	remaining: 53.5s
80:	learn: 0.7915895	total: 1m 50s	remaining: 25.9s
99:	learn: 0.7984749	total: 2m 16s	remaining: 0us

F1 обучающей выборки: 0.798474852197755
F1 тестовой выборки:  0.7527367506516072
Матрица ошибок
[[28270   359]
 [ 1064  2166]]

 Время обработки:
CPU times: total: 7min 56s
Wall time: 2min 41s


In [22]:
result_test['CatBoostRegressor'] = [f1_test]

## Тестирование

### LogisticRegression

In [23]:
%%time
lr_model = LogisticRegression()
lr_model.set_params(**LR_best_params)
lr_model.fit(train_features, train_target)
prediction = lr_model.predict(test_features)
f1 = f1_score(test_target, prediction)

print('F1 тестовой выборки:', f1, '\n')

print('Матрица ошибок')
print(confusion_matrix(test_target, prediction))

print('\n', 'Время обработки:')

F1 тестовой выборки: 0.7677752293577982 

Матрица ошибок
[[27561  1068]
 [  552  2678]]

 Время обработки:
CPU times: total: 10.4 s
Wall time: 5.96 s


In [24]:
result_test['LogisticRegression'] = [f1]


### LGBMClassifier

In [25]:
%%time
LightGBM_model = LGBMClassifier()
LightGBM_model.set_params(**LGBM_best_params)
LightGBM_model.fit(train_features, train_target)
prediction = LightGBM_model.predict(test_features)
f1 = f1_score(test_target, prediction)
print('F1 тестовой выборки:', f1, '\n')

print('Матрица ошибок')
print(confusion_matrix(test_target, prediction))

print('\n', 'Время обработки:')

F1 тестовой выборки: 0.6751491053677932 

Матрица ошибок
[[28527   102]
 [ 1532  1698]]

 Время обработки:
CPU times: total: 2min 44s
Wall time: 53.8 s


In [26]:
result_test['LGBMClassifier'] = [f1]

## Выводы

In [27]:
df_result_test = pd.DataFrame.from_dict(data=result_test, 
                                        columns=['f1'], orient='index')
df_result_test

,f1
CatBoostRegressor,0.752737
LogisticRegression,0.767775
LGBMClassifier,0.675149


И так, барабанная дробь. Побеждает модель LogisticRegression с результатом 0.767775

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны